In [1]:
# import python file from parent folder
from img_cap_lib import *
# imports
import torch
import torchvision
import torchtext
from torchtext.vocab import vocab, GloVe, Vectors
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import os
from PIL import Image
import string
from collections import OrderedDict, Counter
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
import pickle
import os
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import nltk

In [2]:
# check if google colab is installed
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except:
    print("Not using google colab")

Not using google colab


# Modellpfad definieren

In [2]:
model_path = "tiny_model_with_normalisation.pt"

# Daten herunterladen

In [3]:
data_download("flickr8k_tiny")

Data already exi sts at flickr8k_tiny


# Preprocessing

In [4]:
# caption preprocessing
embedding_dim = 300
min_frequency = 1

captions = pd.read_csv("flickr8k_tiny/captions.txt")
caption_preprocessor = CaptionPreprocessor(captions=captions, captions_path="flickr8k_tiny/captions.txt", embedding_dim=embedding_dim, min_frequency=min_frequency)
caption_preprocessor.preprocess()

# image preprocessing
img_preprocessor = ImagePreprocessor(image_size=(112,112), normalize=True, image_folder_path="flickr8k_tiny")
img_preprocessor.preprocess_images()

Shape captions: (40460, 2)


/Users/ronnyschneeberger/Documents/FHNW/HS22/del-image-captioning/img_cap_lib.py:108: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.captions.caption = self.captions.caption.apply(lambda x: x.strip("."))


Shape captions after filtering: (39749, 3)
Removed Captions:  711 , in Percent:  1.76
transformed_images folder already exists. No preprocessing necessary.


# Data Split

In [17]:
batch_size = 64

training_data, test_data = train_test_split(caption_preprocessor.captions, test_size=0.15, random_state=42)

embedding = Embedding(embedding_matrix=caption_preprocessor.embedding, vocabulary=caption_preprocessor.vocabulary)

# create dataset
train_dataset = FlickrDataset(captions=training_data, embedding=embedding, image_folder="flickr8k_tiny")
test_dataset = FlickrDataset(captions=test_data, embedding=embedding, image_folder="flickr8k_tiny")

# create dataloader
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

# Modell erstellen

In [18]:
# create model
# encoder = EncoderCNN(net=torchvision.models.convnext_tiny, pretrained_weights=torchvision.models.ConvNeXt_Tiny_Weights.IMAGENET1K_V1, output_size=300)
# decoder = DecoderRNN(input_size=300, hidden_size=caption_preprocessor.embedding_dim, num_layers=1, dropout=0.0, len_vocab=len(embedding.words))
# embedding = Embedding(embedding_matrix=caption_preprocessor.embedding, vocabulary=caption_preprocessor.vocabulary)
# model = ImageCaptioning(encoder=encoder, decoder=decoder, embedding=embedding, batch_size=batch_size)

model_stats = torch.load("models/tiny_model_with_normalisation.pt", map_location=torch.device(device))
model = load_captioning_model(model_stats=model_stats)

# train model
model_stats = model.train_model(loader=train_loader, optimizer=torch.optim.Adam(model.parameters(), lr=0.001), criterion=torch.nn.CrossEntropyLoss(), epochs=1, print_every=20)

# save model
torch.save(model_stats, model_path)

KeyboardInterrupt: 

# Evaluierung

In [ ]:
class Evaluator:
    def __init__(self, model, dataloader, device):
        # initiate variables 
        self.model = model
        self.dataloader = dataloader
        self.device = device
    
    def evaluate(self):
        scores = []

        for i, (images, captions, lengths, vectorized_captions) in enumerate(self.dataloader):
            # move to device
            images = images.to(self.device)
            captions = captions.to(self.device)
            vectorized_captions = vectorized_captions.to(self.device)
            
            # forward pass
            output = self.model.forward(images)
            references = self.model.words[vectorized_captions.cpu()]

            for j in range(output.shape[0]):
                candidate = self.output_to_sentence(output[j,:])
                reference = self.output_to_sentence(references[j,:])
                scores.append(self.bleu_score(candidate, reference))
            
            print(f"Batch: {i+1} of {len(self.dataloader)}")

        print(f"Average BLEU score: {np.mean(scores)}")
        return np.mean(scores), scores

    @staticmethod
    def output_to_sentence(output:list):
        '''
        Removes Tokens from model output.
        '''
        output = [token for token in output if token not in ["<SOS>", "<EOS>", "<PAD>"]]
        return output

    @staticmethod
    def bleu_score(reference, candidate):
        '''
        Calculates the BLEU score for a single reference and candidate. Uses the SmoothingFunction for smoothing when no overlap between certain n-grams is found. 

        Params:
        -------
        reference: list of strings - The reference sentence.
        candidate: list of strings - The candidate sentence.

        Returns:
        --------
        bleu_score: float - The BLEU score.
        '''
        # calculate the BLEU score
        return nltk.translate.bleu_score.sentence_bleu(reference, candidate, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method1)

In [ ]:
path = model_path
model_stats = torch.load(path, map_location=device)
model = load_captioning_model(model_stats)

In [ ]:
train_dataset = FlickrDataset(captions=training_data, embedding=embedding, image_folder="flickr8k_tiny")
test_dataset = FlickrDataset(captions=test_data, embedding=embedding, image_folder="flickr8k_tiny")

# create dataloader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True, drop_last=True)

# calculate bleu scores
train_evaluator = Evaluator(model, train_loader, device)
test_evaluator = Evaluator(model, test_loader, device)

train_bleu, train_scores = train_evaluator.evaluate()
test_bleu, test_scores = test_evaluator.evaluate()

print(f"Train BLEU: {train_bleu}")
print(f"Test BLEU: {test_bleu}")

In [ ]:
# export bleu scores
with open("tiny_train_scores.pkl", "wb") as f:
    pickle.dump(train_scores, f)

with open("tiny_test_scores.pkl", "wb") as f:
    pickle.dump(test_scores, f)